우선 bert 모델을 편집할 수 있는지 확인해보고자 한다.

In [1]:
import os
os.chdir("/workspace/")

from model.custom_bert_model import InfiniBertModel
from model.speaker_clustering import SpeakerClusterModel
from transformers import BertTokenizer
from transformers import BertConfig
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# BERT 사전학습 모델

BERT 사전학습 모델을 불러오는 코드

HuggingFace BERT를 수정하여 infini-attention 추가

infini BERT에 사전학습 가중치가 로드되는지 확인

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_config = BertConfig.from_pretrained("bert-base-uncased")
bert_config.test = "test"
model = InfiniBertModel.from_pretrained("bert-base-uncased", config = bert_config)
config = model.config
print(config)

Some weights of InfiniBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.9.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.7.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.11.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.10.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.1.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.5.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.3.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.8.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.6.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.0.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.2.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.4.attention.self.fastweight_mem.head_gates']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "test": "test",
  "transformers_version": "4.32.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [34]:
config.max

20

In [3]:
tokenizer.model_max_length

512

In [5]:
tokenizer.sep_token_id

102

In [12]:
tokenizer.cls_token

'[CLS]'

In [7]:
text = "What Does the Sea Say to Shore"
encoded_input = tokenizer(text, return_tensors='pt')

In [8]:
encoded_input

{'input_ids': tensor([[ 101, 2054, 2515, 1996, 2712, 2360, 2000, 5370,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [68]:
model.encoder.init_memories()

In [111]:
output = model.forward(**encoded_input)

In [70]:
output.last_hidden_state

tensor([[[-0.0676,  0.1169, -0.0588,  ..., -0.2462,  0.0030,  0.2178],
         [ 0.3927,  0.0140, -0.3862,  ...,  0.7322, -0.1591, -0.4914],
         [ 0.5550,  0.2541,  0.3846,  ..., -0.0214, -0.2755,  0.4120],
         ...,
         [ 0.2774, -0.0327,  0.2194,  ..., -0.0293,  0.0623,  0.7460],
         [-0.3367, -0.3397, -0.3121,  ..., -0.1021,  0.2643, -0.0252],
         [ 0.8737,  0.2135, -0.4063,  ...,  0.0282, -0.5712, -0.4220]]],
       grad_fn=<NativeLayerNormBackward0>)

In [71]:
tokenizer.cls_token_id

101

In [72]:
text1 = "What Does the Sea Say to Shore"
text2 = "What should we do?"
result = tokenizer.batch_encode_plus([text1, text2])

In [73]:
result

{'input_ids': [[101, 2054, 2515, 1996, 2712, 2360, 2000, 5370, 102], [101, 2054, 2323, 2057, 2079, 1029, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}

In [74]:
output.last_hidden_state.shape

torch.Size([1, 9, 768])

# speaker cluster 모델 실험

speaker cluster 모델 구현

speaker cluster 모델 잘 작동하는지 실험

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_config = BertConfig.from_pretrained("bert-base-uncased")
bert_config.decoder_intermediate_size = 1524
bert_config.feature_dimension = 500

model = SpeakerClusterModel(bert_config)

Some weights of InfiniBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.11.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.5.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.10.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.2.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.8.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.1.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.7.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.4.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.3.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.6.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.0.attention.self.fastweight_mem.head_gates', 'bert.encoder.layer.9.attention.self.fastweight_mem.head_gates']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [3]:
def concat_batch_encode_result(encoded_result: dict):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    for ids in encoded_result["input_ids"]:
        input_ids.extend(ids)
    
    for type_ids in encoded_result["token_type_ids"]:
        token_type_ids.extend(type_ids)
    
    for masks in encoded_result["attention_mask"]:
        attention_mask.extend(masks)
        
    return {
        "input_ids": torch.Tensor([input_ids]).int(), 
        "token_type_ids": torch.Tensor([token_type_ids]).int(),
        "attention_mask": torch.Tensor([attention_mask]).int()
    }

In [4]:
text1 = "What Does the Sea Say to Shore"
text2 = "What should we do?"
result = tokenizer.batch_encode_plus([text1, text2])

In [5]:
concat_result = concat_batch_encode_result(result)

In [6]:
concat_result

{'input_ids': tensor([[ 101, 2054, 2515, 1996, 2712, 2360, 2000, 5370,  102,  101, 2054, 2323,
          2057, 2079, 1029,  102]], dtype=torch.int32),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=torch.int32)}

In [7]:
result = model(**concat_result)

In [10]:
result[0].shape

torch.Size([1, 16, 500])

# 데이터셋 구성

데이터가 생긴 꼴을 살피고, 데이터셋 파일을 구성한다.

In [3]:
import os
os.chdir("/workspace/")

import pandas as pd
import pickle as pkl
import csv
import json

In [4]:
listofnovels = pd.read_csv('data/pdnc/ListOfNovels.txt')
listofnovels

,novel,author,folder
0,A Room With A View,E. M. Forster,ARoomWithAView
1,The Age of Innocence,Edith Wharton,AgeOfInnocence
2,Alice's Adventures in Wonderland,Lewis Carroll,AliceInWonderland
3,Anne Of Green Gables,Lucy Maud Montgomery,AnneOfGreenGables
4,Daisy Miller,Henry James,DaisyMiller
5,Emma,Jane Austen,Emma
6,A Handful Of Dust,Evelyn Waugh,HandfulOfDust
7,Howards End,E. M. Forster,HowardsEnd
8,Night And Day,Virginia Woolf,NightAndDay
9,Northanger Abbey,Jane Austen,NorthangerAbbey


In [5]:
novels = []
with open('data/pdnc/ListOfNovels.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        novels.append(row[-1])

In [6]:
for novel in novels:
    print(novel)

    qdf = pd.read_csv('data/pdnc/novels/'+novel+'/quotations.csv', index_col=0)
    charDict = pkl.load(open('data/pdnc/novels/'+novel+'/charDict.pkl', 'rb'))
    
    print("Number of characters: {}".format(len(charDict['id2names'])))
                             
    with open('data/pdnc/novels/'+novel+'/text.txt', 'r') as f:
        ntext = f.read().strip()
        
    

ARoomWithAView
Number of characters: 67
AgeOfInnocence
Number of characters: 55
AliceInWonderland
Number of characters: 51
AnneOfGreenGables
Number of characters: 115
DaisyMiller
Number of characters: 10
Emma
Number of characters: 18
HandfulOfDust
Number of characters: 106
HowardsEnd
Number of characters: 57
NightAndDay
Number of characters: 54
NorthangerAbbey
Number of characters: 20
Persuasion
Number of characters: 35
PrideAndPrejudice
Number of characters: 82
SenseAndSensibility
Number of characters: 25
SignOfFour
Number of characters: 35
TheAwakening
Number of characters: 22
TheGambler
Number of characters: 27
TheInvisibleMan
Number of characters: 31
TheManWhoWasThursday
Number of characters: 31
TheMysteriousAffairAtStyles
Number of characters: 30
ThePictureOfDorianGray
Number of characters: 43
TheSportOfTheGods
Number of characters: 38
TheSunAlsoRises
Number of characters: 51


In [7]:
qdf

,qTextArr,qSpan,qText,speaker,addressee,qType,refExp,menTexts,menSpans,menEnts
qId,,,,,,,,,,
Q0,['I know a girl in Strasbourg who can show us ...,"[[6412, 6465]]",I know a girl in Strasbourg who can show us th...,Jake Barnes,"['Frances Clyne', 'Robert Cohn']",Anaphoric,I said,[['us']],"[[[6453, 6455]]]","[[['Jake Barnes', 'Robert Cohn']]]"
Q1,"[""She's been there two years and knows everyth...","[[6554, 6654]]",She's been there two years and knows everythin...,Jake Barnes,"['Frances Clyne', 'Robert Cohn']",Anaphoric,I thought it was accidental and went on,[[]],[[]],[[]]
Q2,"['Hell,', 'why go to Strasbourg? We could go u...","[[6777, 6782], [6793, 6860]]","Hell, why go to Strasbourg? We could go up to ...",Jake Barnes,"['Frances Clyne', 'Robert Cohn']",Anaphoric,I said,"[[], ['We']]","[[], [[6815, 6817]]]","[[], [['Jake Barnes', 'Robert Cohn']]]"
Q3,"[""For God's sake,"", ""why did you say that abou...","[[7015, 7030], [7042, 7121]]","For God's sake, why did you say that about tha...",Robert Cohn,['Jake Barnes'],Anaphoric,he said,"[[], ['you', 'you', 'Frances']]","[[], [[7050, 7053], [7105, 7108], [7113, 7120]]]","[[], [['Jake Barnes'], ['Jake Barnes'], ['Fran..."
Q4,"['No, why should I? If I know an American girl...","[[7125, 7226]]","No, why should I? If I know an American girl t...",Jake Barnes,['Robert Cohn'],Implicit,NaN,[['Frances']],"[[[7218, 7225]]]",[[['Frances Clyne']]]
...,...,...,...,...,...,...,...,...,...,...
Q2605,"['Want to go for a ride?', 'Want to ride throu...","[[363401, 363423], [363434, 363464]]",Want to go for a ride? Want to ride through th...,Jake Barnes,['Brett Ashley'],Anaphoric,I said,"[[], []]","[[], []]","[[], []]"
Q2606,"['Right,', ""I haven't seen Madrid. I should se...","[[363468, 363474], [363489, 363532]]","Right, I haven't seen Madrid. I should see Mad...",Brett Ashley,['Jake Barnes'],Explicit,Brett said,"[[], []]","[[], []]","[[], []]"
Q2607,"[""I'll finish this,""]","[[363536, 363553]]","I'll finish this,",Jake Barnes,['Brett Ashley'],Anaphoric,I said,[[]],[[]],[[]]


In [8]:
qdf["qType"].value_counts()

qType
Implicit     1556
Explicit      597
Anaphoric     456
Name: count, dtype: int64

In [11]:
qdf["qType"].value_counts().index[0]

'Implicit'

In [40]:
json.loads(qdf.iloc[0]["qSpan"])[0]

[6412, 6465]

In [27]:
ntext[6412:6465]

'I know a girl in Strasbourg who can show us the town,'

In [35]:
paragraphs = ntext.split('\n')
paragraphs = [p for p in paragraphs if len(p) > 0]
len(paragraphs)

6785

In [36]:
len(qdf)

2610

## Dataset 구현을 위한 실험 코드

In [28]:
import pathlib

In [29]:
novels_dir = pathlib.Path("data/pdnc/novels/")

In [32]:
for dirs in novels_dir.iterdir():
    print("Dirs: ")
    print(dirs)
    print("")

Dirs: 
data/pdnc/novels/AgeOfInnocence

Dirs: 
data/pdnc/novels/SenseAndSensibility

Dirs: 
data/pdnc/novels/DaisyMiller

Dirs: 
data/pdnc/novels/HowardsEnd

Dirs: 
data/pdnc/novels/TheAwakening

Dirs: 
data/pdnc/novels/NightAndDay

Dirs: 
data/pdnc/novels/TheSunAlsoRises

Dirs: 
data/pdnc/novels/SignOfFour

Dirs: 
data/pdnc/novels/NorthangerAbbey

Dirs: 
data/pdnc/novels/AliceInWonderland

Dirs: 
data/pdnc/novels/TheMysteriousAffairAtStyles

Dirs: 
data/pdnc/novels/Emma

Dirs: 
data/pdnc/novels/PrideAndPrejudice

Dirs: 
data/pdnc/novels/ThePictureOfDorianGray

Dirs: 
data/pdnc/novels/ARoomWithAView

Dirs: 
data/pdnc/novels/TheGambler

Dirs: 
data/pdnc/novels/TheSportOfTheGods

Dirs: 
data/pdnc/novels/TheInvisibleMan

Dirs: 
data/pdnc/novels/TheManWhoWasThursday

Dirs: 
data/pdnc/novels/HandfulOfDust

Dirs: 
data/pdnc/novels/Persuasion

Dirs: 
data/pdnc/novels/AnneOfGreenGables

